# Security Agent

# NetFlow

In [1]:
import os
import pandas as pd
import datetime
import calendar
import numpy as np
from sklearn.preprocessing import *
import pickle
import time

In [2]:
import tensorflow as tf
import keras as kr

/home/ahmed/Programs/anaconda3/envs/py27/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras import backend as K
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.regularizers import l2

In [4]:
def create_new_experiment_dir(exp_base_dir):
    def get_unique_name(): return time.strftime("%Y%m%d-%H%M%S")

    exp_base_dir = os.path.abspath(os.path.expanduser(exp_base_dir))
    if exp_base_dir[-1] != '/':
        exp_base_dir += '/'

    # Create new experiment directory
    exp_dir = exp_base_dir + get_unique_name() + '/'
    os.makedirs(exp_dir)

    os.makedirs(exp_dir + 'pickle_dump/')
    os.makedirs(exp_dir + 'logs/')

    return exp_dir

### F1 Score

In [5]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

### Confusion Matrix

In [6]:
def print_confusion_matrix(plabels,tlabels):
    """
        functions print the confusion matrix for the different classes
        to find the error...
        
        Input:
        -----------
        plabels: predicted labels for the classes...
        tlabels: true labels for the classes
        
        code from: http://stackoverflow.com/questions/2148543/how-to-write-a-confusion-matrix-in-python
    """
    import pandas as pd
    plabels = pd.Series(np.squeeze(plabels))
    tlabels = pd.Series(np.squeeze(tlabels))
    
    # draw a cross tabulation...
    df_confusion = pd.crosstab(tlabels,plabels, rownames=['Actual'], colnames=['Predicted'], margins=True)
    
    #print df_confusion
    return df_confusion

# Preprocessing Data

#### Load clean data

In [7]:
data_df = pd.read_csv('./data/netflow_clean.csv')
feat = list(data_df.columns)
feat.remove('label')

#### Ignoring encoding for now

#### One hot encode categorical

In [8]:
# categ_feat = ['source_port', 'destination_port', 'protocol']

# data_df = data.itervalues().next()
# data_df = pd.get_dummies(data_df, prefix=['sp', 'dp', 'pr'], columns=categ, sparse=True)
# data_df

In [9]:
# enc = OneHotEncoder()
# enc.fit(data_df)
# transformed = enc.transform(data_df)

#### Scaling data

#### Dropping features having very close max and min values to avoid NaNs on scaling

In [10]:
diff = data_df[feat].max(axis=0) - data_df[feat].min(axis=0)
eps = 0.00000000001

feat = list(data_df[feat].columns[~(diff.abs() < eps)])

## Dropping Categorical features Source port , Destination port and protocol for now

In [11]:
feat = filter(lambda x: x not in ['source_port', 'destination_port', 'protocol'], feat)

In [12]:
# Scaling only Numerical features
num_feat = feat

# Using MinMaxScaler
min_max = MinMaxScaler()
# Fit scaler
min_max.fit(data_df[num_feat].as_matrix())
# Transform data
data_df[num_feat] = pd.DataFrame(min_max.transform(
    data_df[num_feat].as_matrix()), columns=num_feat)

## Converting multiclass problem to binary classification

In [13]:
data_df['label'][data_df['label'] != 'BENIGN'] = 'ATTACK'

/home/ahmed/Programs/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


#### Encoding labels

In [14]:
le = LabelEncoder()
le.fit(data_df['label'].as_matrix())
data_df['label'] = pd.DataFrame(le.transform(
    data_df['label'].as_matrix()), columns=['label'])

#### Splitting up data

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X = data_df[feat].as_matrix()
y = data_df['label'].as_matrix()

In [17]:
# Splitting Train Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, stratify=y)

/home/ahmed/Programs/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [18]:
# Splitting Train Validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, train_size=0.8, stratify=y_train)

In [19]:
print 'Train data:',(X_train.shape)
print 'Train labels:', (y_train.shape)
print 'Validation data:', (X_val.shape)
print 'Validation labels:', (y_val.shape)
print 'Test data:', (X_test.shape)
print 'Test labels', (y_test.shape)

Train data: (1700892, 69)
Train labels: (1700892,)
Validation data: (425224, 69)
Validation labels: (425224,)
Test data: (531529, 69)
Test labels (531529,)


In [20]:
labels = dict(zip(list(le.classes_), le.transform(list(le.classes_))))
classes = len(labels.keys())
labels

{'ATTACK': 0, 'BENIGN': 1}

In [21]:
print 'Training data:'
for k, v in labels.iteritems():
    print k, 'flows count:', (y_train == v).sum()
print '\n'

print 'Validation data:'
for k, v in labels.iteritems():
    print k, 'flows count:', (y_val == v).sum()
print '\n'

print 'Testing data:'
for k, v in labels.iteritems():
    print k, 'flows count:', (y_test == v).sum()
print '\n'

Training data:
BENIGN flows count: 1401254
ATTACK flows count: 299638


Validation data:
BENIGN flows count: 350314
ATTACK flows count: 74910


Testing data:
BENIGN flows count: 437892
ATTACK flows count: 93637




### Attack vs Benign

In [22]:
print "Benign flows: %.2f%%" % (
    ((data_df['label'] == labels['BENIGN']).sum() / float(data_df.shape[0])) * 100)
print "Attack flows: %.2f%%" % (
    ((data_df['label'] != labels['BENIGN']).sum() / float(data_df.shape[0])) * 100)

Benign flows: 82.38%
Attack flows: 17.62%


### One hot encode labels to use with softmax

In [23]:
label_one_hot_enc = OneHotEncoder(n_values=classes)
y_train_enc = label_one_hot_enc.fit_transform(y_train.reshape(-1, 1)).toarray()
y_val_enc = label_one_hot_enc.transform(y_val.reshape(-1, 1)).toarray()
y_test_enc = label_one_hot_enc.transform(y_test.reshape(-1, 1)).toarray()

# Building the Model

In [36]:
# Training parameters
train_params = {}

train_params['batch_size'] = 1024 # 64 128 256 1024
train_params['epochs'] = 100
train_params['feat'] = X_train.shape[1]
train_params['classes'] = classes
train_params['alpha'] = 0.01
train_params['decay'] = 0.0005
train_params['loss'] = 'categorical_crossentropy'
train_params['optimizer'] = 'Adam'
train_params['reg'] = 0 # 0.001
train_params['dropout'] = 0.4 # Dropout rate

In [37]:
# Keras sequential model
model = Sequential()

model.add(Dense(128, input_shape=(
    train_params['feat'], ), kernel_initializer='he_normal',
    kernel_regularizer=l2(train_params['reg'])))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(train_params['dropout']))

model.add(Dense(256, kernel_initializer='he_normal',
                kernel_regularizer=l2(train_params['reg'])))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(train_params['dropout']))

model.add(Dense(128, kernel_initializer='he_normal',
                kernel_regularizer=l2(train_params['reg'])))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(train_params['dropout']))

model.add(Dense(64, kernel_initializer='he_normal',
                kernel_regularizer=l2(train_params['reg'])))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(train_params['dropout']))

model.add(Dense(32, kernel_initializer='he_normal',
                kernel_regularizer=l2(train_params['reg'])))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(train_params['dropout']))

model.add(Dense(16, kernel_initializer='he_normal',
                kernel_regularizer=l2(train_params['reg'])))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(train_params['dropout']))

model.add(Dense(8, kernel_initializer='he_normal',
                kernel_regularizer=l2(train_params['reg'])))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(train_params['dropout']))

model.add(Dense(train_params['classes'], kernel_initializer='VarianceScaling'))
model.add(BatchNormalization())
model.add(Activation('softmax'))

adam = Adam(lr=train_params['alpha'], decay=train_params['decay'])
model.compile(loss=train_params['loss'], optimizer=adam, metrics=[f1])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 128)               8960      
_________________________________________________________________
batch_normalization_25 (Batc (None, 128)               512       
_________________________________________________________________
activation_25 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 256)               33024     
_________________________________________________________________
batch_normalization_26 (Batc (None, 256)               1024      
_________________________________________________________________
activation_26 (Activation)   (None, 256)               0         
__________

# New experiment

In [38]:
exp_base_dir = './experiments'
exp_dir = create_new_experiment_dir(exp_base_dir)
print 'New experiment directory:', exp_dir
pickle_dir = exp_dir + 'pickle_dump/'
log_dir = exp_dir + 'logs/'

print "Enter experiment summary. Enter empty line or Ctrl-D to save it."
exp_summ = []
while True:
    try:
        line = raw_input("")
    except EOFError:
        break
    if line == '':
        break
    exp_summ.append(line)
exp_summ = '\n'.join(exp_summ)
exp_summ += '\n'

# Save min_max_scaler for used for numerical features
with open(pickle_dir + 'min_max_data.p', 'wb') as f:
    pickle.dump(min_max, f)
# Save label_encoder
with open(pickle_dir + 'label_encoder.p', 'wb') as f:
    pickle.dump(le, f)
# Save label_one_hot_encoder
with open(pickle_dir + 'label_one_hot_encoder.p', 'wb') as f:
    pickle.dump(label_one_hot_enc, f)
# Save tuple of form (final data_df's feature's list, labels)
with open(pickle_dir + 'features_labels.p', 'wb') as f:
    pickle.dump((list(data_df.columns), labels), f)
# Save summaries
with open(exp_dir + 'model_summary', 'w') as f:
    model.summary(print_fn=lambda x: f.write(x + '\n'))
with open(exp_dir + 'experiment_summary', 'w') as f:
    f.write(exp_summ)
# Save Training parameters
with open(exp_dir + 'train_params', 'w') as f:
    for k, v in train_params.iteritems():
        f.write(k + ': ' + str(v) + '\n')

New experiment directory: /home/ahmed/intern_projects/network_security/experiments/20180122-164944/
Enter experiment summary. Enter empty line or Ctrl-D to save it.
dropout 0.4



# Training

In [39]:
# Training checkpoints
checkpoint = ModelCheckpoint(
    filepath=(exp_dir + 'model.h5'), verbose=1, save_best_only=True)
# Tensorboard logs
tensorboard = TensorBoard(log_dir=log_dir, batch_size=train_params['batch_size'])
# Train
model.fit(X_train, y_train_enc, batch_size=train_params['batch_size'], epochs=train_params['epochs'],
          validation_data=(X_val, y_val_enc), callbacks=[checkpoint, tensorboard])

Train on 1700892 samples, validate on 425224 samples
Epoch 1/100
1699840/1700892 [============================>.] - ETA: 0s - loss: 0.1924 - f1: 0.9233
Epoch 00001: val_loss improved from inf to 0.19017, saving model to /home/ahmed/intern_projects/network_security/experiments/20180122-164944/model.h5
1700892/1700892 [==============================] - 34s 20us/step - loss: 0.1924 - f1: 0.9233 - val_loss: 0.1902 - val_f1: 0.9217
Epoch 2/100
1699840/1700892 [============================>.] - ETA: 0s - loss: 0.1509 - f1: 0.9365
Epoch 00002: val_loss improved from 0.19017 to 0.13904, saving model to /home/ahmed/intern_projects/network_security/experiments/20180122-164944/model.h5
1700892/1700892 [==============================] - 33s 19us/step - loss: 0.1509 - f1: 0.9365 - val_loss: 0.1390 - val_f1: 0.9338
Epoch 3/100
1699840/1700892 [============================>.] - ETA: 0s - loss: 0.1423 - f1: 0.9391
Epoch 00003: val_loss improved from 0.13904 to 0.10577, saving model to /home/ahmed/inte

Epoch 27/100
1699840/1700892 [============================>.] - ETA: 0s - loss: 0.1179 - f1: 0.9492
Epoch 00027: val_loss did not improve
1700892/1700892 [==============================] - 33s 19us/step - loss: 0.1179 - f1: 0.9492 - val_loss: 0.0900 - val_f1: 0.9612
Epoch 28/100
1699840/1700892 [============================>.] - ETA: 0s - loss: 0.1177 - f1: 0.9494
Epoch 00028: val_loss did not improve
1700892/1700892 [==============================] - 33s 19us/step - loss: 0.1177 - f1: 0.9494 - val_loss: 0.0900 - val_f1: 0.9608
Epoch 29/100
1699840/1700892 [============================>.] - ETA: 0s - loss: 0.1173 - f1: 0.9492
Epoch 00029: val_loss improved from 0.08955 to 0.08944, saving model to /home/ahmed/intern_projects/network_security/experiments/20180122-164944/model.h5
1700892/1700892 [==============================] - 33s 19us/step - loss: 0.1173 - f1: 0.9492 - val_loss: 0.0894 - val_f1: 0.9611
Epoch 30/100
1699840/1700892 [============================>.] - ETA: 0s - loss: 0.1

KeyboardInterrupt: 

### Experiment notes

In [40]:
print "Enter experiment summary. Enter empty line or Ctrl-D to save it."
exp_summ = []
while True:
    try:
        line = raw_input("")
    except EOFError:
        break
    if line == '':
        break
    exp_summ.append(line)
exp_summ = '\n'.join(exp_summ)
exp_summ += '\n'

with open(exp_dir + 'experiment_summary', 'a') as f:
    f.write(exp_summ)

Enter experiment summary. Enter empty line or Ctrl-D to save it.
val loss dropped to 0.089
f1 score went up to 0.96



# Testing

In [41]:
scores = model.evaluate(X_test, y_test_enc)
with open(exp_dir + 'test_results', 'w') as f:
    for i in xrange(len(scores)):
        s = "%s: %.2f" % (model.metrics_names[i], scores[i])
        print s
        f.write(s + '\n')

531529/531529 [==============================] - 54s 101us/step
loss: 0.09
f1: 0.96


## Test Set Confusion Matrix

In [42]:
y_pred = model.predict(X_test, batch_size=X_test.shape[0], verbose=1)
y_pred = y_pred.nonzero()[1]

531529/531529 [==============================] - 1s 1us/step


In [43]:
confusion_matrix = print_confusion_matrix(
    le.inverse_transform(y_pred), le.inverse_transform(y_test))
confusion_matrix.to_csv(exp_dir + 'confusion_matrix.csv')
confusion_matrix

/home/ahmed/Programs/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ahmed/Programs/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Predicted  ATTACK  BENIGN     All
Actual                           
ATTACK      46798   46839   93637
BENIGN     218967  218925  437892
All        265765  265764  531529